# Tarea 2 - NLP CIMAT: Minería de Texto Básica
## Por: Gustavo Hernández Angeles

## 1. Preparación (Lectura de corpus, tokenización)

In [185]:
import numpy as np
import nltk
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_fscore_support
import pandas as pd

### Leyendo corpus
* Utilizaré parte del código visto en la práctica.

In [186]:
def get_texts_from_file(path_corpus, path_labels):
    tr_txt = []
    tr_y = []

    with open(path_corpus, "r", encoding="utf-8-sig") as f_corpus, open(path_labels, "r", encoding="utf-8-sig") as f_truth:
        for twitt in f_corpus:
            tr_txt += [twitt.strip()]
        for label in f_truth:
            tr_y += [label.strip()]

    return tr_txt, tr_y

In [187]:
tr_txt, tr_y = get_texts_from_file("./data/mex_train.txt", "./data/mex_train_labels.txt")

In [188]:
val_txt, val_y = get_texts_from_file("./data/mex_val.txt", "./data/mex_val_labels.txt")

* Tokenizacion de Corpus  
Haré que solo lea las palabras y quitaré los stopwords

In [189]:
tokenizer = RegexpTokenizer(r'[a-zA-ZáéíóúñÁÉÍÓÚÑ]+')

In [190]:
stopwords_espanol = stopwords.words("spanish")
corpus_palabras = []
for doc in tr_txt:
    text = tokenizer.tokenize(doc)
    corpus_palabras += [w.lower() for w in text if w.lower() not in stopwords_espanol]

fdist = nltk.FreqDist(corpus_palabras)
print(f"Tamaño del vocabulario: {len(fdist)}")

Tamaño del vocabulario: 12161


* Se hace un filtrado por frecuencia, al hacer esto, reducimos el tamaño del vocabulario de 14268 a 4683  
Se escoge un umbral de 1, valor mayor reduce más el vocabulario.  
Con 2 -> el tamaño de V es 2749  
Con 3 -> el tamaño de V es 1973

In [191]:
K_umbral = 1
V = [(fdist[key], key) for key in fdist if fdist[key] > K_umbral] # Más de una ocurrencia
V.sort()
V.reverse()
print(f"Tamaño del vocabulario: {len(V)}")

Tamaño del vocabulario: 4281


In [192]:
dict_indices = dict()
cont = 0
for freq, word in V:
    dict_indices[word] = cont
    cont += 1

## 2. Bolsas de Palabras, Bigramas y Emociones

### 1. Evalúe BOW con pesado binario.

* Definimos una función para realizar el BOW con peso binario.

In [193]:
def binario_bow(tr_txt, V, dict_indices):
    BOW = np.zeros(shape=(len(tr_txt), len(V)), dtype=int)
    
    cont_doc = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        
        for word in fdist_doc:
            # Se elige ignorar si un término no está en el vocab.
            if word not in dict_indices:
                continue
            BOW[cont_doc, dict_indices[word]] = 1
            
        cont_doc += 1
    
    return BOW

* Definimos una función para evaluar el SVM dados las BOW de train y test.

In [194]:
def evaluar_bow(BOW_train, BOW_test, val_y):
    # Parámetro de complejidad del SVM, se proponen estos
    # y se recorrerán con GridSearch
    parameters = {"C": [.05, .12, .25, .5, 1, 2, 4]}
    # Tratar de penalizar con base a la proporción de ejemplos
    # en cada clase
    svr = svm.LinearSVC(class_weight='balanced', max_iter=10000)
    grid = GridSearchCV(estimator=svr, param_grid=parameters,
                        n_jobs=6, scoring="f1_macro", cv=5)
    grid.fit(BOW_train, tr_y)
    y_pred = grid.predict(BOW_test)

    print(confusion_matrix(val_y, y_pred))
    print(metrics.classification_report(val_y, y_pred))
    print(f"F1-score: {f1_score(val_y, y_pred, pos_label='1'):.4f}")
    
    return grid

#### Evaluamos con pesado binario

In [195]:
BOW_train = binario_bow(tr_txt, V, dict_indices)
BOW_test = binario_bow(val_txt, V, dict_indices)
svm_binario = evaluar_bow(BOW_train,BOW_test, val_y)

[[363  55]
 [ 50 119]]
              precision    recall  f1-score   support

           0       0.88      0.87      0.87       418
           1       0.68      0.70      0.69       169

    accuracy                           0.82       587
   macro avg       0.78      0.79      0.78       587
weighted avg       0.82      0.82      0.82       587

F1-score: 0.6939


### 2. Evalúe BOW con pesado frecuencia

In [196]:
# Definimos BOW con esquema de peso de frecuencia.
def frecuencia_bow(tr_txt, V, dict_indices):
    BOW = np.zeros(shape=(len(tr_txt), len(V)), dtype=int)
    
    cont_doc = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        
        for word, freq in fdist_doc.items():
            if word in dict_indices:
                BOW[cont_doc, dict_indices[word]] = freq
        cont_doc += 1
    return BOW

#### Evaluamos con pesado frecuencia

In [197]:
BOW_train = frecuencia_bow(tr_txt, V, dict_indices)
BOW_test = frecuencia_bow(val_txt, V, dict_indices)
svm_frecuencia = evaluar_bow(BOW_train,BOW_test, val_y)

[[358  60]
 [ 51 118]]
              precision    recall  f1-score   support

           0       0.88      0.86      0.87       418
           1       0.66      0.70      0.68       169

    accuracy                           0.81       587
   macro avg       0.77      0.78      0.77       587
weighted avg       0.81      0.81      0.81       587

F1-score: 0.6801


### 3. Evalúe BOW con pesado tf-idf

In [198]:
# Definimos BOW con esquema de peso de tf-idf
def tfidf_bow(tr_txt, V, dict_indices):

    
    # Un BOW de frecuencia nos ayuda a obtener tf
    # Lo haremos logaritmico
    f_bow = frecuencia_bow(tr_txt, V, dict_indices)
    tf = np.where(f_bow > 0, 1 + np.log10(f_bow),0)
    
    # De misma forma para df
    n_docs = len(tr_txt)
    df = np.count_nonzero(f_bow, axis=0)
    idf = np.log10(n_docs / (1+df))
    
    BOW = tf*idf

    return BOW

In [199]:
BOW_train = tfidf_bow(tr_txt, V, dict_indices)
BOW_test = tfidf_bow(val_txt, V, dict_indices)
svm_tfidf = evaluar_bow(BOW_train,BOW_test, val_y)

/tmp/ipykernel_26137/143517032.py:8: RuntimeWarning: divide by zero encountered in log10
  tf = np.where(f_bow > 0, 1 + np.log10(f_bow),0)


[[351  67]
 [ 57 112]]
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       418
           1       0.63      0.66      0.64       169

    accuracy                           0.79       587
   macro avg       0.74      0.75      0.75       587
weighted avg       0.79      0.79      0.79       587

F1-score: 0.6437


### 4. Evalúe BOW con pesado binario normalizado l2

In [200]:
# Función para crear BOW con pesado binario normalizado l2 (norma euclidiana)
def binarionorm_bow(tr_txt, V, dict_indices):
    BOW = binario_bow(tr_txt, V, dict_indices)
    norma = np.linalg.norm(BOW, axis=1, keepdims=True)
    norma[norma==0] = 1 # Evitar div por 0
    return BOW / norma

In [201]:
BOW_train = binarionorm_bow(tr_txt, V, dict_indices)
BOW_test = binarionorm_bow(val_txt, V, dict_indices)
svm_binnorm = evaluar_bow(BOW_train,BOW_test, val_y)

[[355  63]
 [ 50 119]]
              precision    recall  f1-score   support

           0       0.88      0.85      0.86       418
           1       0.65      0.70      0.68       169

    accuracy                           0.81       587
   macro avg       0.77      0.78      0.77       587
weighted avg       0.81      0.81      0.81       587

F1-score: 0.6781


### 5. Evalúe BOW con pesado frecuencia normalizado l2

In [202]:
# Función BOW pesado frecuencia normalizado l2
def frecuencianorm_bow(tr_txt, V, dict_indices):
    BOW = frecuencia_bow(tr_txt, V, dict_indices)
    norma = np.linalg.norm(BOW, axis=1, keepdims=True)
    norma[norma==0] = 1
    return BOW/norma

In [203]:
BOW_train = frecuencianorm_bow(tr_txt, V, dict_indices)
BOW_test = frecuencianorm_bow(val_txt, V, dict_indices)
svm_frecnorm = evaluar_bow(BOW_train,BOW_test, val_y)

[[351  67]
 [ 47 122]]
              precision    recall  f1-score   support

           0       0.88      0.84      0.86       418
           1       0.65      0.72      0.68       169

    accuracy                           0.81       587
   macro avg       0.76      0.78      0.77       587
weighted avg       0.81      0.81      0.81       587

F1-score: 0.6816


### 6. Evalúe BOW con pesado tfidf normalizado l2

In [204]:
def tfidfnorm_bow(tr_txt, V, dict_indices):
    BOW = tfidf_bow(tr_txt, V, dict_indices)
    norma = np.linalg.norm(BOW, axis=1, keepdims=True)
    norma[norma==0] = 1
    return BOW/norma

In [205]:
BOW_train = tfidfnorm_bow(tr_txt, V, dict_indices)
BOW_test = tfidfnorm_bow(val_txt, V, dict_indices)
svm_tfidfnorm = evaluar_bow(BOW_train,BOW_test, val_y)

/tmp/ipykernel_26137/143517032.py:8: RuntimeWarning: divide by zero encountered in log10
  tf = np.where(f_bow > 0, 1 + np.log10(f_bow),0)


[[358  60]
 [ 50 119]]
              precision    recall  f1-score   support

           0       0.88      0.86      0.87       418
           1       0.66      0.70      0.68       169

    accuracy                           0.81       587
   macro avg       0.77      0.78      0.78       587
weighted avg       0.82      0.81      0.81       587

F1-score: 0.6839


### 7. Ponga una tabla comparativa a modo de resumen con las seis entradas anteriores.

Representación  | Macro F1
----------------|----------
Binario         | **69.39%**
Frecuencia      | 68.01%
TF-IDF          | 64.37%
Binario L2      | 67.81%
Frecuencia L2   | 68.16%
TF-IDF L2       | 68.39%


### 8. De las configuraciones anteriores eliga la mejor y evalúela con más y menos términos

#### Todo el vocabulario con al menos dos ocurrencias

In [206]:
V = [(fdist[key], key) for key in fdist if fdist[key] > 1]
V.sort()
V.reverse()
longitud_vocabulario = len(V)
print(f"Tamaño del vocabulario: {longitud_vocabulario}")

dict_indices = dict()
cont = 0
for freq, word in V:
    dict_indices[word] = cont
    cont += 1
print(f"Tamaño del diccionario: {len(dict_indices)}")

Tamaño del vocabulario: 4281
Tamaño del diccionario: 4281


In [207]:
BOW_train = binario_bow(tr_txt, V, dict_indices)
BOW_test = binario_bow(val_txt, V, dict_indices)
svm_masPalabras = evaluar_bow(BOW_train,BOW_test, val_y)

[[363  55]
 [ 50 119]]
              precision    recall  f1-score   support

           0       0.88      0.87      0.87       418
           1       0.68      0.70      0.69       169

    accuracy                           0.82       587
   macro avg       0.78      0.79      0.78       587
weighted avg       0.82      0.82      0.82       587

F1-score: 0.6939


#### Mitad del Vocabulario

In [208]:
V_dosTercios = V[:int(longitud_vocabulario/2)]
dict_indices = dict()
cont = 0
for freq, word in V_dosTercios:
    dict_indices[word] = cont
    cont += 1
print(f"Tamaño del vocabulario: {len(V_dosTercios)}")
print(f"Tamaño del diccionario: {len(dict_indices)}")

Tamaño del vocabulario: 2140
Tamaño del diccionario: 2140


In [209]:
BOW_train = binario_bow(tr_txt, V_dosTercios, dict_indices)
BOW_test = binario_bow(val_txt, V_dosTercios, dict_indices)
svm_mitadPalabras = evaluar_bow(BOW_train,BOW_test, val_y)

[[360  58]
 [ 47 122]]
              precision    recall  f1-score   support

           0       0.88      0.86      0.87       418
           1       0.68      0.72      0.70       169

    accuracy                           0.82       587
   macro avg       0.78      0.79      0.79       587
weighted avg       0.82      0.82      0.82       587

F1-score: 0.6991


#### 1000 palabras

In [210]:
V_1000 = V[:1000]
dict_indices = dict()
cont = 0
for freq, word in V_1000:
    dict_indices[word] = cont
    cont += 1
print(f"Tamaño del vocabulario: {len(V_1000)}")
print(f"Tamaño del diccionario: {len(dict_indices)}")

Tamaño del vocabulario: 1000
Tamaño del diccionario: 1000


In [211]:
BOW_train = binario_bow(tr_txt, V_dosTercios, dict_indices)
BOW_test = binario_bow(val_txt, V_dosTercios, dict_indices)
svm_1000 = evaluar_bow(BOW_train,BOW_test, val_y)

[[358  60]
 [ 51 118]]
              precision    recall  f1-score   support

           0       0.88      0.86      0.87       418
           1       0.66      0.70      0.68       169

    accuracy                           0.81       587
   macro avg       0.77      0.78      0.77       587
weighted avg       0.81      0.81      0.81       587

F1-score: 0.6801


#### Recurso Léxico EmoLex para construir una Bolsa de Emociones (BoE)

In [212]:
emolex_path = "./data/Spanish-NRC-EmoLex.txt"
emolex = pd.read_csv(emolex_path, sep="\t")

El recurso que encontré se encuentra en `./data/Spanish-NRC-EmoLex.txt`. Inspeccionandolo encuentro que:
* Cada renglón es una palabra, al que le corresponde un *vector de emociones* incluyendo el sentimiento **positivo** o **negativo**.
* Las palabras provienen del inglés y son traducidas al español, por lo que puede haber palabras repetidas.

In [213]:
emolex.head()

,English Word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,Spanish Word
0,aback,0,0,0,0,0,0,0,0,0,0,detrás
1,abacus,0,0,0,0,0,0,0,0,0,1,ábaco
2,abandon,0,0,0,1,0,1,0,1,0,0,abandonar
3,abandoned,1,0,0,1,0,1,0,1,0,0,abandonado
4,abandonment,1,0,0,1,0,1,0,1,1,0,abandono


Quitaré las dimensiones `positivo`, `negativo` y `English Word`.

In [214]:
clean_emolex = emolex.drop(columns=["English Word", "positive", "negative"])
clean_emolex.head()

,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,Spanish Word
0,0,0,0,0,0,0,0,0,detrás
1,0,0,0,0,0,0,0,1,ábaco
2,0,0,0,1,0,1,0,0,abandonar
3,1,0,0,1,0,1,0,0,abandonado
4,1,0,0,1,0,1,1,0,abandono


Siguiendo la propuesta de enmascarar cada término con su emoción, pienso representar cada tweet mediante la suma de emociones de sus términos.


In [ ]:
def tweetAVector(tweet : str, emolex : pd.DataFrame):
    vector = np.zeros(8)
    palabras = tokenizer.tokenize(tweet.lower())
    
    # Obtener solo palabras en el lexicon
    ocurrencias = emolex[emolex["Spanish Word"].isin(palabras)]
    
    if not ocurrencias.empty:
        vector = ocurrencias.drop(columns="Spanish Word").sum(axis=0).to_numpy()
    
    return vector
    #return vector / np.linalg.norm(vector) if np.linalg.norm(vector) != 0 else vector

In [216]:
def crearBoE(tr_txt, emolex):
    BoE = np.zeros(shape=(len(tr_txt), 8))
    
    for i, tweet in enumerate(tr_txt):
        BoE[i] = tweetAVector(tweet, emolex)
        
    return BoE

In [217]:
bag_of_emotions_train = crearBoE(tr_txt, clean_emolex)
bag_of_emotions_test = crearBoE(val_txt, clean_emolex)

In [184]:
svm_emolex = evaluar_bow(bag_of_emotions_train, bag_of_emotions_test, val_y)

/home/gus/anaconda3/envs/tf_env/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[[358  60]
 [123  46]]
              precision    recall  f1-score   support

           0       0.74      0.86      0.80       418
           1       0.43      0.27      0.33       169

    accuracy                           0.69       587
   macro avg       0.59      0.56      0.57       587
weighted avg       0.65      0.69      0.66       587

F1-score: 0.3345
